In [ ]:
#################### Params ###############

NOVA_HOME = '/home/labs/hornsteinlab/Collaboration/NOVA_GAL/NOVA'
preprocessing_path = "/home/labs/hornsteinlab/Collaboration/FUNOVA/outputs/preprocessing/brenner"
exp = '_exp4'
csv_name = f'raw_metrics110225{exp}.csv'
mappings_filepath = f"/home/labs/hornsteinlab/Collaboration/NOVA_GAL/NOVA/manuscript/markers_focus_boundries/markers_focus_boundries_funova{exp}_25.2.25.csv"
imgs_path = '/home/labs/hornsteinlab/Collaboration/FUNOVA/input/images/raw/'

metric_name = 'Target_Sharpness_Brenner'
img_shape = 1024
percentiles_resolution = 0.0001
percentile_ranges_for_reports = [0, 0.1, 0.2, 0.3, 0.5, 0.7, 1, 2, 5, 10, 15, 20, 30, 40, 60, 75, 80, 85, 90, 95, 98, 99, 99.5, 99.7, 99.8,99.9,100]

In [ ]:
import os
import sys
import concurrent.futures
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec


os.environ['NOVA_HOME'] = '/home/projects/hornsteinlab/Collaboration/MOmaps'

sys.path.insert(1, os.getenv('NOVA_HOME'))
sys.path.insert(1, os.getenv("NOVA_HOME"))
print(f"NOVA_HOME: {os.getenv('NOVA_HOME')}")

from src.preprocessing.preprocessing_utils import get_image_focus_quality 
from src.preprocessing.preprocessing_utils import rescale_intensity, fit_image_shape

%reload_ext autoreload
%autoreload 2
%aimport

%matplotlib inline

# Utils

In [ ]:
def get_metrics(tile, as_string=False):
    sharpness_brenner = get_image_focus_quality(tile)    
    if as_string:
        return f"Brenner: {round(sharpness_brenner, 3)}"
    return sharpness_brenner

def show_images(df, max_samples = 10):
    for ind, path in enumerate(df.Path.values):
        print(ind)
        if max_samples is not None and ind >= max_samples:
            print(f"Stopping at {ind}. There are {len(df.Path.values)} images in total")
            break
        
        # Target
        target_path = os.path.join(imgs_path, path)
        show_processed_tif(target_path)
        # His DAPI
        # path_l = target_path.split("/")
        # path_l[-2] = 'DAPI'
        
        # file_name = path_l[-1].split("_")
        # dapi_file_name = "_".join([file_name[0], 'w1confDAPI', file_name[-1]])
        # dapi_file_name = "/".join([*path_l[:-1], dapi_file_name])
        # print(dapi_file_name)

        # show_processed_tif(dapi_file_name)
        print('--------------------------------')
        
def init_mappings(markers=[], filepath=None):
    if filepath is not None:     
        if os.path.exists(filepath):
            mappings = pd.read_csv(filepath, index_col=0)
            return mappings
        
    mappings = pd.DataFrame(columns=['Lower_bound', 'Upper_bound'], index=markers)

    return mappings
        
def save_to_mapping(filepath, mappings, marker, value, is_upper_bound):
    col = 'Upper_bound' if is_upper_bound else 'Lower_bound' 
    mappings.loc[marker, col] = value
    
    mappings.to_csv(filepath)
    print(f"File saved to {filepath}")
    
    
def processed_path_to_raw_paths(path):
    path_splited = path.split(os.sep)
    filename = path_splited[-1]
    filename_split = filename.split('_')
    rep = filename_split[0]
    panel = filename_split[4]
    line = filename_split[5]
    filename_new = '_'.join(filename_split[1:4])
    batch = path_splited[-5]
    batch = batch.split('_')[0]
    cond = path_splited[-3]
    marker = path_splited[-2]
    
    ret = os.path.join("/home/projects/hornsteinlab/Collaboration/MOmaps/input/images/raw/SpinningDisk", batch, line, panel, cond, rep, marker, f'{filename_new}.tif')
    return ret

def raw_path_to_processed_path(path):
    path_splited = path.split(os.sep)
    filename = os.path.splitext(path_splited[-1])[0]
    rep = path_splited[-3]
    panel = path_splited[-5]
    line = path_splited[-6]
    batch = path_splited[-7]
    batch = f"{batch}_16bit_no_downsample"
    cond = path_splited[-4]
    marker = path_splited[-2]
    
    ret = os.path.join("/home/projects/hornsteinlab/Collaboration/MOmaps/input/images/processed/spd2/SpinningDisk/", batch, line, cond, marker, f'{rep}_{filename}_{panel}_{line}_processed.npy')
    return ret


d = '/home/projects/hornsteinlab/Collaboration/MOmaps/input/images/raw/SpinningDisk/'

def show_label(path):
    path_l = path.split("/")
    return path_l[-7:]

def process_tif(path):
    """
    Read and process the image.

    Parameters:
        path (str): Path to the image file.

    Returns:
        ndarray: Processed image.
    """
    # read the image stack
    img = cv2.imread(path, cv2.IMREAD_ANYDEPTH)
    img = fit_image_shape(img, (img_shape, img_shape))
    # rescale pixel intensities
    img = rescale_intensity(img)
    return img
    
def show_processed_tif(path):
    img = process_tif(path)
    print(get_metrics(img, True))
    # show the image with grid 
    fig, ax = plt.subplots(figsize=(7,7))
    plt.imshow(img, cmap='gray')
    put_tiles_grid(image=img, ax=ax)
    plt.axis('off')
    plt.title(show_label(path), color='purple')
    print(f"Img shape: {img.shape}")
    plt.show()

def put_tiles_grid(image, ax):
    # assumes 1000x1000 image
    import matplotlib.patches as patches

    # Add dashed grid lines for 64 blocks
    num_blocks = 10
    block_size = 100

    for i in range(1, num_blocks):
        # Draw horizontal dashed lines
        ax.plot([0, 1000], [i * block_size, i * block_size], linestyle='--', lw=1, alpha=0.5, color='pink')

        # Draw vertical dashed lines
        ax.plot([i * block_size, i * block_size], [0, 1000], linestyle='--', lw=1, alpha=0.5, color='pink')

    # Remove x and y axis labels
    ax.set_xticks([])
    ax.set_yticks([])

    # Add a title
    plt.title('Image with Dashed Grid of 64 Blocks')

def update_all_mappings(mappings, thresholds, df):
    # Iterate over the rows and fill the thresholds
    for marker in mappings.index:
        if marker in thresholds and thresholds[marker] is not None:
            df_marker = df.loc[df['Marker'] == marker]
            percentiles = df_marker[metric_name].describe(percentiles=percentiles_to_describe)
            mappings.loc[marker, "Lower_bound"] = round(percentiles[f'{thresholds[marker][0]}%'], 2)
            mappings.loc[marker, "Upper_bound"] = round(percentiles[f'{thresholds[marker][1]}%'], 2)
    return mappings

def create_histogram_report_by_batch(df: pd.DataFrame, all_markers: list) -> None:
    """
    Generate a PDF report with histograms for each marker.

    Parameters:
        df (pd.DataFrame): DataFrame containing the images data.
        all_markers (list): List of unique markers.

    Returns:
        None
    """    
    with PdfPages('Marker_histogram_by_batch.pdf') as pdf:
        for marker in all_markers:
            print(marker)
            df_marker = df.loc[df['Marker'] == marker]
            percentiles = df_marker[metric_name].describe(percentiles=percentiles_to_describe)
            create_histogram(
                df_marker, percentiles, 
                low_perc=thresholds[marker][0], 
                high_perc=thresholds[marker][1], 
                overlay_group=['Batch'], 
                x_min=0.1, x_max=99.9
            )
            plt.title(marker)
            pdf.savefig()
            plt.close()
            
def create_histogram(
    df_marker: pd.DataFrame, 
    percentiles: pd.Series, 
    low_perc: float = 0.5, 
    high_perc: float = 99.9, 
    x_min: float = None, 
    x_max: float = None, 
    overlay_group: list = None,
    plot_base: bool = True,
    actual_x_limits: tuple = None
) -> None:
    """
    Create a histogram of the Brenner values of a certain marker.

    Parameters:
        df_marker (pd.DataFrame): Marker's data.
        percentiles (pd.Series): Percentile values for annotations.
        low_perc (float): Low percentile threshold for annotations.
        high_perc (float): High percentile threshold for annotations.
        x_min (float, optional): Minimum x-axis value for the histogram.
        x_max (float, optional): Maximum x-axis value for the histogram.
        overlay_group (list, optional): Columns to group and overlay histograms.
        plot_base (bool): Whether to plot the base histogram.
        actual_x_limits (tuple, optional): Tuple specifying actual x_min and x_max values.

    Returns:
        None
    """
    assert low_perc < high_perc, "'low_perc' must be less than 'high_perc'"
    
    # Determine histogram range
    if actual_x_limits:
        hist_range = actual_x_limits  # Use passed x_min and x_max
    else:
        hist_range = (percentiles[f'{x_min}%'], percentiles[f'{x_max}%']) if x_min is not None and x_max is not None else None
    
    # Plot base histogram
    if plot_base:
        plt.hist(df_marker[metric_name].values, bins=100, range=hist_range, color=plt.cm.tab10(range(1))[0], 
                 alpha=0.3, label='Brenner scores')
    
    # Plot overlays
    if overlay_group is not None:
        grouped_data = df_marker.groupby(overlay_group)
        unique_groups = grouped_data.groups.keys()
        colors = plt.cm.tab10(range(len(unique_groups)))

        for color, group in zip(colors, unique_groups):
            group_data = grouped_data.get_group(group)
            group_label = ' - '.join(map(str, group)) if isinstance(group, tuple) else group
            plt.hist(group_data[metric_name].values, bins=100, range=hist_range, alpha=0.4, label=group_label, color=color)

    # Add percentile markers
    plt.scatter(percentiles['50%'], 0.5, color='yellow', s=12, label='50th percentile')
    plt.scatter(percentiles[f'{high_perc}%'], 0.5, color='orange', s=12, label=f'{high_perc}th percentile')
    plt.scatter(percentiles[f'{low_perc}%'], 0.5, color='red', s=12, label=f'{low_perc}th percentile')

    # Remove duplicate legend entries
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())

def generate_marker_reports(
    df: pd.DataFrame, 
    all_markers: list, 
    output_folder: str, 
    percentiles_to_describe: list, 
    percentile_ranges: list, 
    max_samples: int
) -> None:
    """
    Generate a detailed report for each marker, including histograms and filtered images.

    Parameters:
        df (pd.DataFrame): DataFrame containing the data.
        all_markers (list): List of unique markers.
        output_folder (str): Path to save the output PDFs.
        percentiles_to_describe (list): List of percentiles to describe the metric.
        percentile_ranges (list): List of percentile ranges for filtering.
        max_samples (int): Maximum number of images to display per range.

    Returns:
        None
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for marker in all_markers:
        print(marker)
        df_marker = df.loc[df['Marker'] == marker]
        percentiles = df_marker[metric_name].describe(percentiles=percentiles_to_describe)

        # Define the actual x-axis limits for consistent base and overlay histograms
        actual_x_limits = (percentiles['0%'], percentiles['97%'])

        pdf_path = os.path.join(output_folder, f'output_report_{marker}.pdf')
        with PdfPages(pdf_path) as pdf:
            # Generate histograms with overlays for different groups
            groups = ['Condition', 'Batch', 'Rep', 'CellLine']
            for group in groups:
                create_histogram(
                    df_marker,
                    percentiles,
                    low_perc=0,
                    high_perc=97,
                    actual_x_limits=actual_x_limits,
                    overlay_group=[group]
                )
                plt.title(f"Histogram with Overlay by {group}")
                pdf.savefig()  # Save current figure to the PDF
                plt.close()

            # Combined base histogram and overlay for each cell line
            for CL in np.unique(df_marker['CellLine']):
                create_histogram(
                    df_marker,
                    percentiles,
                    low_perc=0,
                    high_perc=97,
                    actual_x_limits=actual_x_limits,
                )
                
                # Filter and overlay the specific cell line
                df_tmp = df_marker.loc[df_marker['CellLine'] == CL]
                percentiles_tmp = df_tmp[metric_name].describe(percentiles=percentiles_to_describe)
                create_histogram(
                    df_tmp,
                    percentiles,
                    low_perc=0,
                    high_perc=97,
                    actual_x_limits=actual_x_limits,
                    overlay_group=['CellLine', 'Condition'], plot_base = False
                )
                plt.title(f"Histogram for (Cell Line: {CL})")
                # Save the combined plot to the PDF
                pdf.savefig()
                plt.close()

            # Remaining parts of the function (filtered images, percentile ranges, etc.)
            for i in range(len(percentile_ranges) - 1):
                per_min = np.round(percentile_ranges[i], 2)
                per_max = np.round(percentile_ranges[i + 1], 2)
                assert per_min < per_max, "Percentile range minimum must be less than the maximum."
                threshold = percentiles[f'{per_min}%']
                threshold_second = percentiles[f'{per_max}%']

                c = (df_marker[metric_name] >= threshold) & (df_marker[metric_name] <= threshold_second)
                df_marker_filtered = df_marker[c].sample(frac=1, random_state=1)

                text_output = (f'Images between %{per_min} - {per_max}%\n'
                               f"Number of {marker} images in threshold {threshold} "
                               f"({per_min}%) and {threshold_second} ({per_max}%): "
                               f"{len(df_marker_filtered)}\n\n"
                               f"{df_marker_filtered['CellLine'].value_counts().to_string()}\n\n"
                               f"{df_marker_filtered['Condition'].value_counts().to_string()}\n\n")

                fig = plt.figure(figsize=(12, 8))
                gs = GridSpec(3, 1, figure=fig, height_ratios=[1, 2, 0.1])
                text_ax = fig.add_subplot(gs[0, :])
                text_ax.axis('off')
                text_ax.text(0.01, 0.99, text_output, ha='left', va='top', fontsize=12, wrap=True)

                filtered_paths = df_marker_filtered['Path'].values
                num_images = min(max_samples, len(filtered_paths))
                img_gs = gs[1].subgridspec(1, num_images, wspace=0.1)

                for ind, path in enumerate(filtered_paths[:num_images]):
                    target_path = os.path.join(output_folder, path)
                    img = process_tif(target_path)

                    ax = fig.add_subplot(img_gs[0, ind])
                    ax.imshow(img, cmap='gray')
                    put_tiles_grid(image=img, ax=ax)
                    ax.axis('off')

                    labels = show_label(path)
                    perc_brenner = abs(percentiles[[per for per in percentiles.keys() if '%' in per]] - get_image_focus_quality(img)).idxmin()
                    ax.set_title(f"{labels[1]}, {labels[3]}, {get_metrics(img, True)}, {perc_brenner}", color='purple', fontsize=10)

                plt.tight_layout()
                pdf.savefig(fig)
                plt.close(fig)

# Main

In [ ]:
# df = pd.read_csv("/home/labs/hornsteinlab/Collaboration/MOmaps_Sagy/MOmaps/sandbox/outliers_detection/raw_metrics_all_batches_all_metrics_site_fix.csv")
# df = pd.read_csv("/home/labs/hornsteinlab/Collaboration/MOmaps_Sagy/MOmaps/sandbox/outliers_detection/raw_metrics_all_batches_brenner_site_dNLS.csv")
# df = pd.read_csv("/home/labs/hornsteinlab/Collaboration/MOmaps/sandbox/outliers_detection/brenner_values/raw_metrics_fus_fixed200224_2.csv")

# df = pd.read_csv("/home/labs/hornsteinlab/Collaboration/MOmaps/outputs/preprocessing/spd/brenner/raw_metrics250324.csv")
# df = pd.read_csv("/home/labs/hornsteinlab/Collaboration/MOmaps/outputs/preprocessing/Opera/brenner/raw_metrics250324.csv")

df = pd.read_csv("/home/projects/hornsteinlab/Collaboration/MOmaps/outputs/preprocessing/Opera18Days/brenner/raw_metrics210524.csv")

df

In [ ]:
df["Marker"] = df["Marker"].str.replace("_", "-")

In [ ]:
df.Path.iloc[0]

In [ ]:
df['Marker'].value_counts()

In [ ]:
counts = df['Marker'].value_counts()
all_markers = df['Marker'].unique()
print(len(all_markers))
print(all_markers)

In [ ]:
mappings_filepath = "/home/projects/hornsteinlab/Collaboration/MOmaps/manuscript/markers_focus_boundries/markers_focus_boundries_opera18days.csv"
mappings = init_mappings(markers=all_markers, filepath=mappings_filepath)
mappings

In [ ]:
percentiles_to_describe = np.arange(0, 1+percentiles_resolution, percentiles_resolution)

# Create Brenner reports

Run this block if you want to generate Brenner reports

In [ ]:
output_folder = f"{preprocessing_path}/brenner_reports{exp}_{datetime.now().strftime('%Y-%m-%d')}"
print('output_folder is', output_folder)
generate_marker_reports(df, all_markers, output_folder, percentiles_to_describe, percentile_ranges_for_reports, max_samples=3)

# Examine and set Brenner one by one (Option 1)

In [ ]:
#################### SAFE ZONE TO CHANGE ###############

marker = 'DAPI' # MAKRENAME
# [marker] Options:
# 'Autophagy' 'DAPI' 'impaired_Autophagosome' 'UPR_ATF4' 'UPR_ATF6'
#  'UPR_IRE1a' 'Ubiquitin_levels' 'DNA_damage_P53BP1' 'Neuronal_activity'
#  'Necroptosis_HMGB1' 'Necrosis' 'DNA_damage_pH2Ax' 'Parthanatos_early'
#  'Cytoskeleton' 'Stress_initiation' 'mature_Autophagosome'
#  'Nuclear_speckles_SON' 'TDP-43' 'Nuclear_speckles_SC35'
#  'Splicing_factories' 'Aberrant_splicing' 'Parthanatos_late'
#  'Protein_degradation' 'Senescence_signaling' 'Apoptosis'
#  'Necroptosis_pMLKL'


# [per] Options: 0-100
per = 98 # percentile threshold 

# [per] Options: 0-100
# *Optional! if you want to view images between per and another threshold
per_second_bound = None

max_samples = 3 # set max number of images (in threshold) to show

# [is_upper_bound] Options:
# True: upper bound
# False: lower bound
is_upper_bound = True 

show_percentile_plot = False

###################### END OF SAFE ZONE ###################





#########################################

if per_second_bound is None:
    per_second_bound = 100 if is_upper_bound else 0

        
print(f"marker = {marker}, per: {per}% (per_second_bound={per_second_bound}%), max_samples = {max_samples}, is upper bound: {is_upper_bound}")


df_marker = df.loc[df['Marker'] == marker]
percentiles = df_marker[metric_name].describe(percentiles=percentiles_to_describe)

threshold = percentiles[f'{per}%']
threshold_second = percentiles[f'{per_second_bound}%']

if is_upper_bound:
    c = (df_marker[metric_name]>=threshold) & (df_marker[metric_name]<=threshold_second)
else:
    c = (df_marker[metric_name]<=threshold) & (df_marker[metric_name]>=threshold_second) 

# threshold
df_marker_filtered = df_marker[c]
# shuffle
df_marker_filtered = df_marker_filtered.sample(frac=1, random_state=1)

print(f"Number of {marker} images in threshold {threshold} ({per}%) (and {threshold_second} ({per_second_bound}%)): {len(df_marker_filtered)}")
print("\n\n")
print(df_marker_filtered['CellLine'].value_counts().to_string())
print(df_marker_filtered['Condition'].value_counts().to_string())

if show_percentile_plot:
    plt.figure(figsize=(15,6))
    plt.plot(percentiles.keys().to_numpy()[4:-1], percentiles.values[4:-1])
    plt.ylabel('value')
    plt.xlabel('percentile')
    plt.xticks(rotation=90)
    plt.show()

show_images(df_marker_filtered, max_samples=max_samples)    

save_to_mapping(mappings_filepath, mappings, marker, round(threshold,2), is_upper_bound)


# Examine Brenners and set the threshold in the next block (option 2)

1. Examine Brenners (write the thresholds in the next block)

In [ ]:
#################### SAFE ZONE TO CHANGE ###############

marker = 'Neuronal-activity' # MAKRENAME
# [marker] Options:
# ['Autophagy' 'DAPI' 'impaired-Autophagosome' 'UPR-ATF4' 'UPR-ATF6'
#  'UPR-IRE1a' 'Ubiquitin-levels' 'DNA-damage-P53BP1' 'Neuronal-activity'
#  'Necroptosis-HMGB1' 'Necrosis' 'DNA-damage-pH2Ax' 'Parthanatos-early'
#  'Cytoskeleton' 'Stress-initiation' 'mature-Autophagosome'
#  'Nuclear-speckles-SON' 'TDP-43' 'Nuclear-speckles-SC35'
#  'Splicing-factories' 'Aberrant-splicing' 'Parthanatos-late'
#  'Protein-degradation' 'Senescence-signaling' 'Apoptosis'
#  'Necroptosis-pMLKL']

per_min = 89
per_max = 90
max_samples = 20

###################### END OF SAFE ZONE ###################



df_marker = df.loc[df['Marker'] == marker]
percentiles = df_marker[metric_name].describe(percentiles=percentiles_to_describe)

print(f'Showing images between %{per_min} - {per_max}')
threshold = percentiles[f'{per_min}%']
threshold_second = percentiles[f'{per_max}%']

c = (df_marker[metric_name]>=threshold) & (df_marker[metric_name]<=threshold_second) 

# threshold
df_marker_filtered = df_marker[c]
# shuffle
df_marker_filtered = df_marker_filtered.sample(frac=1, random_state=1)
# df_marker_filtered.index = range(len(df_marker_filtered))

print(f"Number of {marker} images in threshold {threshold} ({per_min}%) (and {threshold_second} ({per_max}%)): {len(df_marker_filtered)}")
print("\n")
print(df_marker_filtered['CellLine'].value_counts().to_string())
print("\n")
print(df_marker_filtered['Condition'].value_counts().to_string())
print("\n")
show_images(df_marker_filtered, max_samples=max_samples)    

2. Setting Brenners in the csv

In [ ]:
## Exp3
# thresholds = {
#     "Stress-initiation": (0, 99),
#     "Aberrant-splicing": (1, 97),
#     "Autophagy": (5, 97),
#     "DAPI": (0.2, 95),
#     "Apoptosis": (0.27, 95),
#     "impaired-Autophagosome": (5, 99),
#     "Cytoskeleton": (10, 99.8),
#     "DNA-damage-P53BP1": (0.3, 90),
#     "DNA-damage-pH2Ax": (7, 85),
#     "mature-Autophagosome": (2, 98),
#     "Necrosis": (4, 95),
#     "Neuronal-activity": (5, 90),
#     "Nuclear-speckles-SC35": (0.2, 85),
#     "Nuclear-speckles-SON": (0.2, 85),
#     "Parthanatos-early": (0, 85),
#     "Parthanatos-late": (5, 95),
#     "Protein-degradation": (5, 92),
#     "Senescence-signaling": (10, 90),
#     "Splicing-factories": (0, 95),
#     "TDP-43": (20, 99.2),
#     "Ubiquitin-levels": (2, 90),
#     "UPR-ATF4": (20, 99.7), 
#     "UPR-ATF6": (3, 100),
#     "UPR-IRE1a": (2, 98),
#     "Necroptosis-pMLKL": (5, 90),
#     "Necroptosis-HMGB1": (0.01, 90),
# }
## Exp 4 
thresholds = {
    "Stress-initiation": (0.3, 98),
    "Aberrant-splicing": (2, 97),
    "Autophagy": (0, 98),
    "DAPI": (0.2, 98.3),
    "Apoptosis": (0.5, 95),
    "impaired-Autophagosome": (13, 99.5),
    "Cytoskeleton": (10, 99.9),
    "DNA-damage-P53BP1": (0.3, 92),
    "DNA-damage-pH2Ax": (7, 95),
    "mature-Autophagosome": (5, 98.85),
    "Necrosis": (7.2, 98),
    "Neuronal-activity": (7, 94),
    "Nuclear-speckles-SC35": (0.1, 92),
    "Nuclear-speckles-SON": (0.2, 85),
    "Parthanatos-early": (0, 99),
    "Parthanatos-late": (5, 99),
    "Protein-degradation": (2, 98),
    "Senescence-signaling": (4, 99.75),
    "Splicing-factories": (0, 95),
    "TDP-43": (20, 99.8),
    "Ubiquitin-levels": (0.2, 90),
    "UPR-ATF4": (20, 99.75),
    "UPR-ATF6": (3, 100),
    "UPR-IRE1a": (2, 98),
    "Necroptosis-pMLKL": (2, 95),
    "Necroptosis-HMGB1": (0.2, 90),
}

mappings = update_all_mappings(mappings, thresholds, df)
mappings.to_csv(mappings_filepath)
mappings

In [ ]:
# Print top brenners per marker
max_samples = 5
df_path = "/home/projects/hornsteinlab/Collaboration/MOmaps/outputs/preprocessing/Opera/brenner/raw_metrics250324.csv"

print(f"max_samples = {max_samples}")

df = pd.read_csv(df_path)
print(df.shape)

markers = df['Marker'].unique()
print(markers.shape)

for marker in markers:
    print(marker)
    df_marker = df.loc[df['Marker'] == marker]
    print(f"df_marker shape: {df_marker.shape}")
    df_marker.sort_values('Target_Sharpness_Brenner', ascending=False, inplace=True)
    df_marker_subset = df_marker.iloc[:max_samples]
    show_images(df_marker_subset, max_samples=max_samples)   